In [1]:
import sys
from pathlib import Path
from warnings import warn

import cv2
import h5py as h5
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm

sys.path.append('../lib')
from modeling import models, registry
from modeling.utils import make_layer_hook, recur_collapse_feats
from storage import get_storage_functions
from local_paths import stim_dir, cache_dir

# Parameters

In [2]:
#============================================================================
# image to process
#============================================================================
im_md5s    = 'md5_im1,md5_im2'
sep        = ','
im_w       = 16    # size of full image; ...
im_h       = 16    # unit: dva, but only ratio (im_size/patch_size) really matters
ar_tol     = 3/4   # aspect ratio tolerance (between image file and provided size)


#============================================================================
# patch size and resolution
#============================================================================
patch_size =  2    # size of each crop patch
patch_step =  0.5  # step size of patch location


#============================================================================
# model params
#============================================================================
model_name    = 'vit_large_patch16_384'
layer_name    = 'blocks.13.attn.qkv'
spatial_averaging = True  # over W, H for conv; over S for attention


#============================================================================
# paths
#============================================================================
# unlike other scripts, this one is intentionally unaware of subfolders
# (thereby requiring image IDs, e.g., MD5s, to truly be unique)
# all images are in [stim_dir]/Stimuli), so specify it explicitly
stim_dir = stim_dir + 'Stimuli/'

output_root = cache_dir + 'feats/'


#============================================================================
# misc
#============================================================================
device = 'cuda:0'
bgc = (128,128,128)   # background color; used to pad images

In [3]:
# Parameters
patch_size = 2
patch_step = 0.5
model_name = "vit_large_patch16_384"
layer_name = "blocks.13.attn.qkv"
spatial_averaging = True
sep = ","
device = "cuda:0"
im_md5s = "d832ddba1a68ed403d909f671c9096c3,9aede2ee5dd71134c6ce85a4ccb1de00,ed3690df12e821a98c60cb3f1366eb76,c554811739d11e14e8964e80a41c1876,67f61cf36b10ede882a1fd839ef6a631,f8dcd0868cd4c5aa30078a0d3832bdad,819e4e1407d427e68143f59411e9e055,27cf8fbabe43ea4ec749625cc7daf740,579c7e77160cf6203e8487c66e5f1a34,4b342e870cfb72b5e9f8492394512afb,713ef1e1d00b25d91f4c941d955bafde,3de18c3692e42ec95a2facb20e62fa7f,4aedcacb369fad7a860e14df968fb829,80a9dc8a38fb405a4f2b51aa61de0638,b21a53538ef350ce20ac8af512f6bfc6,8faee146dfb8774847f87d3efb55964c,871338f95474804fe20b2c7d55fc98a2,bcabddd3487228d0b045cfb9a8a048bd,672777e7e0b7e8f3a6803be56d58c887,aa5a88070e1e6da00f2d6a08faa57160,9deaaec1a87bc37a3ddcae03662060ff,c5d1d1603734cb398cd4a12307ff95b9,d3ff2f85446ca5822fb190c0fb2a407f,070a8131161279a5c873adc632fe76ea,013a5a63a2697ee7a4c5b54b21326a75,972c0100ad8f4ba4e41649541ce5849b,35f4d47a009bceaf16bbde1dd62d0444,366fb89beb7ad40885a071c62c715850,1b55e043880c89a9b48168c19da562a5,7d2b0b12c27fa9dd436f6bd23381458d,150a8f24120700b2dce7c4ffeb1f3135,f7593bf9adc31e34c2e10681d1108590,78d28d99674b95dc610bd26c15d13976,6a9282a1cb283cf446eba6b22bea1c7f,301b16b0815c4095e70180b4fccee749,c32a5eec2e8b948aaff59037aed9b827,90990756b041549dac801f3aae2cbfc7,6508762d46ea20481e2c141ae1f5ec13,437264ee010db6260990a332fe9245c6,2295cb3803e575eec763cb4a5cff3bc1,46fa493c48388613701546571e8fe768,69d26d03a7af769999988a82eb08f5ac,cce51db01919f2add04b9b742303bab1,4a7763bd757e47df7a5f3a4ba372a288,f7021463509712221b729695b33dd437,725a825f362893169ec8a4df5bfefb3f,ed322f2d287d92dfef2a5e0c66b1371b,b0ca0c2dd42e43c1c5196ffc6aaf8cc7,fd01a223b2089b8d82654f21c63f2835,b1416beddbab3a4a369a579a4f9398a0,57d64742615f4e297cdd506032fecd90,5e106b513ba775f9c104b5a6ac937bfb,d3c89c954d09e653ce8e5d483aafd3a2,546e584a3df212645aad1037f827c0b4,9b1191bf2ab7967f6a064dc4e7d113be,71bf8d48aeb8bf379368164dd137051e,3678df9e3126fd5d6930e1ad21fdc118,4d987592ec68e97580b078955a352365,391fea5e4fbaf90fb3a9f66663790fd6,0458f647680026908491d97f65103689,a80e4aaa9bc840d1c96607e2e29e3cd5,39021618861746763d07246daaae53bb,e533692c90bb7e4fc499ed14273102b4,00aaec16b3bbec32345ac519f7997b5f,f2e58c1695aa1418c96f1b63e31f0b2d,8b45cbbf8743b6af3adf6ff114368120,d843c46ae7adb11fbec3fd03d576d1b2,2891e4cc65acfd881975519a2f44ebb2,f97c52951bc252cf49fd252c36d4a24a,7b088317e80665c38ba04ebd1a3d341e,a4155bd41a4be35045dea65ba66a4a10,96afa87271eb7e650b9db7d478f1c4d5,1b3beeb96c18384fe9b5252bae5faa32,850e9b1b71cf4eac24ac1b6b82f674b9,4a5bd3dd051ddee71684210675ce8cd4,6d9aa3eaac8fe79859c14654dc9bcaaf,6c5aa4af7fe9efef8768686222a20d6a,7fb07e84978df3176d48ad708e03f272,e70747d0b957db51774f32461903f778,bd2405aeb469d25def3f9516aca8fe82,effb5358523e8f69152fdc07ec04e0ad,d817531aeede14c4995113e5d9d5bd09,a342e6ce79480260712c2df6883cf8ce,54cc63ceaddece4bb4c5ba7d5508177f,1538372f799a5baa48771d5800b36533,4518750062c949110fa1c705de6a2fc5,030c1e8b0a0a6d11ec6181c886831e18,3bb4125546ff6817fc3350972559a2a3,9329b0722674e16c2d251a563d1d7db2,2debf1a5f97191af218dc02308bfff24,252be332b55165b88f5018e0f16551a1,b042d14f2b5192dc7dfb8ff09968682f,4ce117a206b5c1bb9dde77bac3eec37f,8c11c5bf6da7bb99a419e28c55918b67,28d3fdffaa4e10289c00ffff3d7af6df,0f57e5a716cd8e420093c0ffeb4e5edd,bc1302ffba50e0bf354f27d52b86f163,ffbeef2b507d2c9bc102262721650489,bb9c78a8b4e8667cacce3b4161dc814d,d033ee475b3d839d7b2b626407d21047,bc46578612b260853194eec10933a06f,f0e31045d10beed78d179bac09536608,ca2cd6a98eee9855ec667c2b7778f9b1,7033f7f39d3c9a195fbb0f8eead59fc3,a2710e0c72b61388811552d81293c451,fc1997228fe2e50bfd6c58e53d0ecee7,1c7f44b5c1df984dddcde5b04c908f15,4958ca9258b213f709e7130330979b49,7987577c8850e7ad15b88ce80fddd600,afd44d8a854b54de501d007e3fc0bbe5,68a309d97b6ddb61eeeeed3043e0f019,3d6fcb3ba0b641b98ba371fc977efc5d,b5bf8c9bd58028fe29e77926aa2978df,1907ff5860356cbedf475b971b6c2f3c,b89836ead826c1756950b7f672e7765d,cb93f1eb3f20fc2329353a64f45ab4e7,bd590b6cf3d816083abec58d7cc5fd94,b2da629dc01ed9d58f53b2912cefe9d7,338b87a9501b76524a764ee8b695e4bb,1802ef3a875a45a6db82dd9d53f04ad4,9925f5d1990c68c709c441921107d892,f2f90d39178f7da54e9be08b13cc12e2,3059ac398c66aba21d325ec70c916b20,8847df2b3aba1250b20bbe3fbcbf8de3,55ee791b7db2fadff8a1ed824ecc1bea,55a118680cb597f6b1de98902d4bb6e7,604dddeccb0f5d6d779c3b5c6965d419,d43d17a8f9a71d6eb3f2c13465c43a88,236d0ef8a2c476b9186cb4cebc157ff9,f08b3ac353d562dd772c8804174a9d77,397c2377811fd8fb95169ab56d1bb7da,ffe891e25df7ece31fe88948775066d3,a8aed751afe9b65ddaf3c5c998ba41cf,1b82574d59f72fe1baa8aed9e732b36a,b73046a711fe40012f7660207e1794c1,0dcaa79e72c5e482c786b2639f3bc1b6,54db4ac69ceabf02a8331e1baefbcb59,65d8f218b774e5c7b16a683340a1644e,55633e1fc1432eadd443a55d5f5f4714,7c5ea0550dd55306d0d5d767fae609f0,fb0549a9b76943801f04429302417b61,235d5cbf28c7b702210e9e3a6e92b44f,1efb4d116f8e1bc0b9211f7380e65e78,52d57238fec2509d237381cb02f04020"
im_w = 16.0
im_h = 16.0


# Check prereqs and params

In [4]:
print('Loading images from folder', stim_dir)
stim_dir = Path(stim_dir).expanduser()
assert stim_dir.is_dir()

output_root = Path(output_root)
output_path = output_root / model_name / layer_name / \
    f'{im_w:.1f}x{im_h:.1f}_as_{patch_size}x{patch_size}_in_{patch_step:.2f}_steps.h5'
print('Saving results to', output_path)
output_path = output_path.expanduser()
output_path.parent.mkdir(exist_ok=True, parents=True)

Loading images from folder ~/Data/free_viewing/Stimuli/
Saving results to ~/Data/free_viewing/Cache/feats/vit_large_patch16_384/blocks.13.attn.qkv/16.0x16.0_as_2x2_in_0.50_steps.h5


# Prepare parameters; save config

In [5]:
im_md5s = im_md5s.split(sep)
print('Processing', len(im_md5s), 'images')

Processing 144 images


In [6]:
im_size = (im_w, im_h)
ar_tol = min(ar_tol, 1/ar_tol)
patch_step = float(patch_step)

# interpret this as origin == lower left
patches_ledge_x = np.arange(int(np.ceil(im_size[0]/patch_step))) * patch_step  # full and right partial patches
patches_ledge_x = np.concatenate([
    np.arange(-1, -int(np.ceil(patch_size/patch_step)), -1)[::-1] * patch_step,  # left partial patches
    patches_ledge_x])
patches_ledge_x -= im_size[0] / 2  # align to image center
n_patches_x = len(patches_ledge_x)

patches_ledge_y = np.arange(int(np.ceil(im_size[1]/patch_step))) * patch_step  # full and right partial patches
patches_ledge_y = np.concatenate([
    np.arange(-1, -int(np.ceil(patch_size/patch_step)), -1)[::-1] * patch_step,  # left partial patches
    patches_ledge_y])
patches_ledge_y -= im_size[1] / 2  # align to image center
n_patches_y = len(patches_ledge_y)

print('Patches step size:', patch_step)
print(f'Number of patches: {n_patches_x} x {n_patches_y} (x-by-y)')
print('Patches (bin lower edge):')
print('(The coordinates in degrees are with origin at image center)')
print('x:')
print('\t' + str(patches_ledge_x).replace('\n', '\n\t'))
print('y:')
print('\t' + str(patches_ledge_y).replace('\n', '\n\t'))

Patches step size: 0.5
Number of patches: 35 x 35 (x-by-y)
Patches (bin lower edge):
(The coordinates in degrees are with origin at image center)
x:
	[-9.5 -9.  -8.5 -8.  -7.5 -7.  -6.5 -6.  -5.5 -5.  -4.5 -4.  -3.5 -3.
	 -2.5 -2.  -1.5 -1.  -0.5  0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.
	  4.5  5.   5.5  6.   6.5  7.   7.5]
y:
	[-9.5 -9.  -8.5 -8.  -7.5 -7.  -6.5 -6.  -5.5 -5.  -4.5 -4.  -3.5 -3.
	 -2.5 -2.  -1.5 -1.  -0.5  0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.
	  4.5  5.   5.5  6.   6.5  7.   7.5]


In [7]:
model_imsize = registry.get_default_preproc(model_name)['imsize']
print('Model input image size:', model_imsize)

Model input image size: 384


In [8]:
tqdm_ = lambda x: tqdm(x, mininterval=300, miniters=10)  # to avoid bloated log file

In [9]:
save_results, add_attr_to_dset, check_equals_saved, link_dsets, copy_group = \
    get_storage_functions(output_path)

In [10]:
save_results('config/stimuli/size_dva', im_size)

group = 'config/patch_grid/'
save_results(group+'size', patch_size)
save_results(group+'step', patch_step)
save_results(group+'eft_edges', patches_ledge_x)
save_results(group+'right_edges', patches_ledge_x+patch_size)
save_results(group+'lower_edges', patches_ledge_y)
save_results(group+'upper_edges', patches_ledge_y+patch_size)
save_results(group+'x_locs', patches_ledge_x+patch_size/2)
save_results(group+'y_locs', patches_ledge_y+patch_size/2)

group = 'config/modelling/'
save_results(group+'model_name', model_name)
save_results(group+'layer_name', layer_name)
save_results(group+'input_image_size', model_imsize)
save_results(group+'spatial_averaging', spatial_averaging)

# Locate & load images

In [11]:
done_md5s = None
offset = 0
if output_path.is_file():
    with h5.File(output_path, 'r') as f:
        try:
            done_md5s = f['md5'][()].astype(str)
            offset = len(done_md5s)
        except KeyError:
            pass

if done_md5s is not None:
    done_md5s = set(done_md5s)
    print(len(done_md5s), 'images already done')
    im_md5s = [v for v in im_md5s if v not in done_md5s]
    print('Processing', len(im_md5s), 'remaining images')

In [12]:
im_paths = [next(stim_dir.glob(md5+'.*')) for md5 in im_md5s]
assert all(p.is_file() for p in im_paths)

In [13]:
ar = im_size[0] / im_size[1]
print(f'Defined image aspect ratio: {ar:.2f}')

n_ims = len(im_paths)
images = np.empty(n_ims, dtype=object)

for iim, fp in enumerate(im_paths):
    image = Image.open(fp)

    # check image aspect ratio
    ar_ = image.size[0] / image.size[1]
    if not (0.99 < ar_/ar < 1.01):
        if not (ar_tol < ar_/ar < 1/ar_tol):
            warn(
                f'image {fp.name} (size: {image.size}; AR = {ar_:.2f}) '
                f'is very far from expected aspect ratio (size: {im_size}; AR = {ar:.2f} '
                '(resizing it regardless, beceause it would have been presented at the '
                'specified size)')

        i = np.argmin(image.size[:2] / np.array(im_size))
        if i == 0:
            w = image.size[0]
            h = int(round(w / ar))
        else:
            h = image.size[1]
            w = int(round(h * ar))
        print(f'Resizing {fp.name} (size: {image.size}; AR = {ar_:.2f}) to size {(h,w)} (AR = {w/h:.2f})')
        image = np.array(image.resize((w, h)))
    else:
        image = np.array(image)

    # make ims 8-bit RGB
    assert image.dtype == np.uint8
    if image.ndim == 3:
        assert image.shape[-1] in (3,4)
        if image.shape[-1] == 4:
            image = image[:,:,:3]
    else:
        assert image.ndim == 2
        image = np.repeat(image[:,:,None], 3, axis=-1)

    images[iim] = image

print(len(images), 'images')
images.shape, images.dtype

Defined image aspect ratio: 1.00


Resizing effb5358523e8f69152fdc07ec04e0ad.jpg (size: (398, 405); AR = 0.98) to size (398, 398) (AR = 1.00)
Resizing d817531aeede14c4995113e5d9d5bd09.jpg (size: (401, 412); AR = 0.97) to size (401, 401) (AR = 1.00)


Resizing 54db4ac69ceabf02a8331e1baefbcb59.jpg (size: (323, 400); AR = 0.81) to size (323, 323) (AR = 1.00)
144 images


((144,), dtype('O'))

# Prepare model

In [14]:
# when no images to process, save time by not loading model
# (unfortunately, I do not know how to early-stop an ipynb from within itself)
if len(im_md5s):

    model = models.get_model_by_name(model_name, dev=device)
    preprocessing_func = models.get_preprocessor_by_model_name(model_name, preproc_imsize=False, preproc_from='numpy')

    class Embedder:
        def __init__(
                self, model=model, preproc_fun=preprocessing_func,
                model_name=model_name, layer_names=layer_name,
                spatial_averaging=spatial_averaging,
                fwd_fun=None, device=device, pbar=tqdm_):

            self.model = model
            self.preproc_fun = preproc_fun
            self.spatial_averaging = spatial_averaging
            self.device = device
            self.pbar = pbar

            if isinstance(layer_names, str):
                layer_names = (layer_names,)
            else:
                assert all(isinstance(n, str) for n in layer_names)
            self.layer_names = layer_names

            if fwd_fun is None:
                if model_name is not None and 'CLIP' in model_name:
                    fwd_fun = model.encode_image
                else:
                    fwd_fun = model.__call__
            self.fwd_fun = fwd_fun

            hooks = {}
            hdls = {}
            for n in layer_names:
                hooks[n], hdls[n] = make_layer_hook(model, n, return_handle=True)
            self.hooks = hooks
            self.hdls = hdls

        def extract_pooled_features(self, ims):
            feats = {n: [] for n in self.layer_names}
            with torch.no_grad():
                for im in self.pbar(ims):
                    tim = self.preproc_fun(im).unsqueeze(0).to(self.device)
                    self.fwd_fun(tim)

                    for n, hook in self.hooks.items():
                        feats_ = recur_collapse_feats(hook.o, spatial_averaging=self.spatial_averaging)
                        if not isinstance(feats_, torch.Tensor):
                            raise ValueError(f'unexpected feature type {type(feats_)} at layer {n}: {feats_}')
                        feats_ = feats_.cpu().numpy()
                        feats[n].append(feats_)

            return {n: np.array(v) for n, v in feats.items()}

In [15]:
if len(im_md5s):
    embedder = Embedder()
    test_im = np.full((model_imsize, model_imsize, 3), bgc, dtype=np.uint8)

    feats = embedder.extract_pooled_features([test_im])
    feats = feats[layer_name][0]
    print('feats:', feats.shape, feats.dtype)
    sample_feats = feats
    feats_shape = sample_feats.shape
    save_results('config/modelling/pooled_feat_shape', sample_feats.shape)

    with h5.File(output_path, 'a') as f:
        if 'feats/bg' not in f:
            save_results('feats/bg', sample_feats)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.25it/s]

feats: (3072,) float32


# Initialize result storage

In [16]:
def create_ignoring_existing(f, *args, attrs=None, **kwargs):
    assert isinstance(args[0], str)
    try:
        dset = f.create_dataset(*args, **kwargs)
        if attrs is not None:
            assert isinstance(attrs, dict)
            for k, v in attrs.items():
                dset.attrs[k] = v
    except ValueError as e:
        if 'name already exists' not in str(e):
            raise
        dset = f[args[0]]
        if attrs is not None:
            assert isinstance(attrs, dict)
            for k, v in attrs.items():
                check_equals_saved(v, dset.attrs[k], k)

In [17]:
cache_opts = dict(compression='gzip', compression_opts=9)
if len(im_md5s):
    with h5.File(output_path, 'a') as f:
        create_ignoring_existing(
            f, 'md5',
            shape=(0,),
            maxshape=(None,),
            chunks=(1,),
            dtype='S32',
            **cache_opts)

        dims = np.array(['image', 'feat_chan'], dtype=bytes)
        coords = np.array(['md5', 'feat_chans'], dtype=bytes)
        create_ignoring_existing(
            f, 'feats/full_image',
            shape=(0,)+feats_shape,
            maxshape=(None,)+feats_shape,
            attrs=dict(dims=dims, coords=coords),
            chunks=(1,)+feats_shape,
            dtype=sample_feats.dtype,
            **cache_opts)

        dims = np.array(['image', 'rf_x', 'rf_y', 'feat_chan'], dtype=bytes)
        coords = np.array(['md5', 'patch_locs', 'patch_locs', 'feat_chans'], dtype=bytes)
        shape_ = (n_patches_x, n_patches_y,) + feats_shape
        create_ignoring_existing(
            f, 'feats/patch_grid',
            shape=(0,)+shape_,
            maxshape=(None,)+shape_,
            attrs=dict(dims=dims, coords=coords),
            chunks=(1,)+shape_,
            dtype=sample_feats.dtype,
            **cache_opts)

# Main loop

In [18]:
def get_image_patch(im, im_size_dva, patch_min_x_dva, patch_min_y_dva, wsize_dva, wsize_px, bgc=bgc):
    assert isinstance(im, np.ndarray)# and im.shape[0] == im.shape[1]
    assert isinstance(wsize_px, int)
    map1 = np.arange(wsize_px)
    map2 = map1.copy()
    ppd = im.shape[0] / im_size_dva[0]
    map1 = (
        ppd * (map1+0.5) / wsize_px * wsize_dva
        + ppd * (patch_min_x_dva + im_size_dva[0] / 2)
    ).astype(np.float32)
    ppd = im.shape[1] / im_size_dva[1]
    map2 = (
        ppd * (map2+0.5) / wsize_px * wsize_dva
        + ppd * (-patch_min_y_dva -wsize_dva + im_size_dva[1] / 2)
    ).astype(np.float32)
    map1 = np.repeat(map1[None,:], wsize_px, 0)
    map2 = np.repeat(map2[:,None], wsize_px, 1)
    wim = cv2.remap(
        im.astype(np.float32), map1, map2, interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT, borderValue=bgc)
    wim = np.round(wim).astype(np.uint8)
    return wim

In [19]:
embedder.pbar = iter  # to avoid bilayer tqdm
for iim, (im, md5) in enumerate(zip(tqdm_(images), im_md5s)):
    i_ = offset + iim

    # full image feats
    im_ = np.array(Image.fromarray(im).resize((model_imsize, model_imsize)))
    feats_ = embedder.extract_pooled_features([im_])
    feats = feats_[layer_name][0]

    with h5.File(output_path, 'a') as f:
        dset = f['feats/full_image']
        if dset.shape[0] < i_ + 1:
            dset.resize(i_+1, axis=0)
        dset[i_] = feats

    # patch grid feats
    featss = []

    for ix, x0 in enumerate(patches_ledge_x):
        featss.append([])

        for iy, y0 in enumerate(patches_ledge_y):
            wim = get_image_patch(im, im_size, x0, y0, patch_size, model_imsize)
            feats_ = embedder.extract_pooled_features([wim])
            feats = feats_[layer_name][0]
            featss[-1].append(feats)

    featss = np.array(featss)

    with h5.File(output_path, 'a') as f:
        dset = f['feats/patch_grid']
        if dset.shape[0] < i_ + 1:
            dset.resize(i_+1, axis=0)
        dset[i_] = featss

        dset = f['md5']
        if dset.shape[0] < i_ + 1:
            dset.resize(i_+1, axis=0)
        dset[i_] = md5

100%|███████████████████████████████████████| 144/144 [1:34:54<00:00, 39.55s/it]

# Wrap up

In [20]:
%load_ext watermark
%watermark
%watermark -vm --iversions -rbg

Last updated: 2024-02-28T16:34:10.309801-05:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.12.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-97-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.12.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.0-97-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 20
Architecture: 64bit

Git hash: 06b6b5e005b560a11dd64bd4e535c8a2dc44f773

Git repo: https://github.com/willwx/free_viewing_staging.git

Git branch: master

numpy: 1.24.3
cv2  : 4.7.0
torch: 2.0.1
h5py : 3.8.0
PIL  : 9.5.0
sys  : 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]

